In [2]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

/Users/alvis/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # Load the tokenizer, model, and data collator
# MODEL_NAME = "google/flan-t5-base"
MODEL_PATH = "/Users/alvis/Study/models/flan-t5-base" # 从本地加载模型

tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [23]:
tokenizer

T5Tokenizer(name_or_path='/Users/alvis/Study/models/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>',

In [24]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## 文本总结

In [13]:
def text_summarization(text):
    # 添加任务指令
    prompts = [
        f"Provide a brief summary of this text: {text}",
        f"TL;DR: {text}",
        f"In summary, the main points are: {text}",
        f"Write a short summary of the following: {text}",
        f"Key points from the text: {text}"
    ]

    for i, prompt in enumerate(prompts):
        print(f"\n尝试提示词 {i+1}: {prompt[:50]}...")
        
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(
            **inputs,
            max_length=100,
            num_beams=4,
            repetition_penalty=1.3,
            early_stopping=True
        )
        
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 如果结果合理就返回
        if len(summary) > 10 and not any(word in summary.lower() for word in ['repeat', 'same']):
            print("结果", summary)
            # return summary
    # return "无法生成合适的摘要"
    
# def text_summarization(text):
#     prompt = f"Summarize the following text in one concise paragraph: {text}"
#     # NT: 容易生成重复的内容
    
#     inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    
#     outputs = model.generate(
#         **inputs,
#         max_length=150,
#         min_length=30,
#         length_penalty=2.0,        # 鼓励生成长度适中的文本
#         num_beams=4,               # 使用集束搜索
#         early_stopping=True,
#         no_repeat_ngram_size=3,    # 防止3-gram重复
#         repetition_penalty=1.5,    # 重复惩罚系数
#         temperature=0.7,           # 降低随机性
#         do_sample=False            # 关闭采样，使用确定性方法
#     )
    
#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary

# 测试文本摘要
text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans. Leading AI textbooks define the field as the study of intelligent agents: any system that perceives its environment and takes actions that maximize its chance of achieving its goals. Some popular accounts use the term artificial intelligence to describe machines that mimic cognitive functions that humans associate with the human mind, such as learning and problem solving.
"""
text_summarization(text)


尝试提示词 1: Provide a brief summary of this text: 
Artificial ...
结果 Artificial Intelligence (AI) is a field of science that has been around for a long time.

尝试提示词 2: TL;DR: 
Artificial intelligence (AI) is intelligen...
结果 Understand artificial intelligence.

尝试提示词 3: In summary, the main points are: 
Artificial intel...
结果 Learn about artificial intelligence.

尝试提示词 4: Write a short summary of the following: 
Artificia...
结果 Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans.

尝试提示词 5: Key points from the text: 
Artificial intelligence...
结果 Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals. Some popular accounts use the term artificial intelligence to describe machines that mimic cognitive functions that humans associate with the human mind.


## 机器翻译

In [16]:
def translate_text(text, source_lang="English", target_lang="Chinese"):
    # 构建翻译指令
    # prompt = f"Translate the following {source_lang} text to {target_lang}: {text}"
    prompts = [
        f"Translate this {source_lang} text to {target_lang}: {text}",
        f"Please translate the following {source_lang} text into {target_lang}: {text}",
        f"Convert this {source_lang} sentence to {target_lang}: {text}",
        f"{source_lang} to {target_lang} translation: {text}",
        f"将以下{source_lang}文本翻译成{target_lang}: {text}"
    ]

    for prompt in prompts:
        print(f"尝试提示词: {prompt}")
        
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(
            **inputs,
            max_length=100,
            min_length=5,  # 设置最小长度避免空输出
            num_beams=4,
            early_stopping=True,
            repetition_penalty=1.2,
            temperature=0.7,
            do_sample=True  # 启用采样以获得更多样化的输出
        )
        
        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 检查结果是否有效
        if translation.strip() and len(translation.strip()) > 2:
            print("翻译结果:", translation)
            # return translation
    
    # return "翻译失败，请尝试其他文本"
# def translate_text(text, source_lang="English", target_lang="Chinese"):
#     # 构建翻译指令
#     prompt = f"Translate the following {source_lang} text to {target_lang}: {text}"
#     inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
#     outputs = model.generate(
#         **inputs,
#         max_length=100,
#         num_beams=4
#     )
    
#     translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return translation

# 测试翻译
english_text = "Hello, how are you today? I hope you're having a wonderful day."
translate_text(english_text)

尝试提示词: Translate this English text to Chinese: Hello, how are you today? I hope you're having a wonderful day.
尝试提示词: Please translate the following English text into Chinese: Hello, how are you today? I hope you're having a wonderful day.
尝试提示词: Convert this English sentence to Chinese: Hello, how are you today? I hope you're having a wonderful day.
尝试提示词: English to Chinese translation: Hello, how are you today? I hope you're having a wonderful day.
尝试提示词: 将以下English文本翻译成Chinese: Hello, how are you today? I hope you're having a wonderful day.
翻译结果: EnglishChinese: Hello, how are you today?


## 问答系统

In [17]:
def question_answering(question, context):
    # 构建问答格式
    prompt = f"Based on the following context, answer the question. Context: {context} Question: {question}"
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# 测试问答
context = "Paris is the capital and most populous city of France. It is located on the Seine River in the north of the country."
question = "What is the capital of France?"
print("问答结果:", question_answering(question, context))

问答结果: Paris


## 文本分类/情感分析 (Sentiment Analysis)

In [7]:
def sentiment_analysis(text):
    prompt = f"Analyze the sentiment of the following text. Is it positive, negative or neutral? Text: {text}"
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=10,
        num_beams=2
    )
    
    sentiment = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sentiment

# 测试情感分析
test_texts = [
    "I absolutely love this product! It's amazing!",
    "This is the worst experience I've ever had.",
    "The package arrived on time and in good condition."
]

for text in test_texts:
    print(f"文本: {text}")
    print(f"情感: {sentiment_analysis(text)}\n")

文本: I absolutely love this product! It's amazing!
情感: positive

文本: This is the worst experience I've ever had.
情感: negative

文本: The package arrived on time and in good condition.
情感: positive



## 文本生成 (Text Generation)

In [18]:
def text_generation(prompt):
    full_prompt = f"Continue writing: {prompt}"
    
    inputs = tokenizer(full_prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=200,
        temperature=0.7,  # 控制创造性
        do_sample=True,   # 启用采样
        top_k=50,         # 限制候选词
        early_stopping=True
    )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 测试文本生成
prompt = "In a distant future where humans have colonized Mars,"
print("生成文本:", text_generation(prompt))

生成文本: Mars will serve as a metaphor for how the human race has changed.
